In [77]:
from book_cover import Book
import os
import requests
from IPython.display import Image
from typing import Optional
import json
from dotenv import load_dotenv
import gradio as gr
from openai import OpenAI

import importlib

def reload_book_class():
    """
    Reload and return the latest definition of Book class from book_cover.py.
    Useful during development in notebooks.
    """
    import book_cover
    importlib.reload(book_cover)
    return book_cover.Book



In [78]:
reload_book_class()

book_cover.Book

In [79]:
load_dotenv(override=True)

False

In [80]:
pushover_user = os.getenv("PUSHOVER_USER")
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_url = "https://api.pushover.net/1/messages.json"


In [81]:
def push(message):
    print(f"Push: {message}")
    payload = {"user": pushover_user, "token": pushover_token, "message": message}
    requests.post(pushover_url, data=payload)

In [82]:
def record_user_details(email, name="Name not provided", notes="not provided"):
    push(f"Recording interest from {name} with email {email} and notes {notes}")
    return {"recorded": "ok"}

In [83]:
def record_unknown_question_about_book(question):
    push(f"Recording {question} asked that I couldn't answer")
    return {"recorded": "ok"}

In [84]:

def find_books(query)-> Optional[list[Book]]:
    books = Book.get_books(query, limit=3)
    
    if books:
        print(f"Found {len(books)} books for {query}")
        return books
    else:
        print(f"No books found for {query}")
        return None


In [85]:
def get_book_cover_image(isbn) -> Optional[Image]:
   result = Book.get_book_cover_image(isbn) 
   return result


In [86]:
find_books_json = {
    "name": "find_books",
    "description": "Always use this tool to find books if the user asks about books or authors (writers)",
    "parameters": {
        "type": "object",
        "properties": {
            "query": {
                "type": "string",
                "description": "query to search the books or writers"
            },
        },
        "required": ["query"],
        "additionalProperties": False
    }
}

In [87]:
get_book_cover_image_json = {
    "name": "get_book_cover_image",
    "description": "Always use this tool to find book's cover image that match the isbn",
    "parameters": {
        "type": "object",
        "properties": {
            "isbn": {
                "type": "string",
                "description": "isbn of the book"
            },
        },
        "required": ["isbn"],
        "additionalProperties": False
    }
}

In [88]:
record_unknown_question_about_book_json = {
    "name": "record_unknown_question_about_book",
    "description": "Always use this tool to record any question about book or author (writer) that couldn't be answered as you didn't know the answer",
    "parameters": {
        "type": "object",
        "properties": {
            "question": {
                "type": "string",
                "description": "The question that couldn't be answered"
            },
        },
        "required": ["question"],
        "additionalProperties": False
    }
}

In [ ]:
tools = [{"type": "function", "function": find_books_json},
        {"type": "function", "function": record_unknown_question_about_book_json}]


In [90]:
tools 

[{'type': 'function',
  'function': {'name': 'find_books',
   'description': 'Always use this tool to find books if the user asks about books or authors (writers)',
   'parameters': {'type': 'object',
    'properties': {'query': {'type': 'string',
      'description': 'query to search the books or writers'}},
    'required': ['query'],
    'additionalProperties': False}}},
 {'type': 'function',
  'function': {'name': 'record_unknown_question_about_book',
   'description': "Always use this tool to record any question about book or author (writer) that couldn't be answered as you didn't know the answer",
   'parameters': {'type': 'object',
    'properties': {'question': {'type': 'string',
      'description': "The question that couldn't be answered"}},
    'required': ['question'],
    'additionalProperties': False}}}]

In [ ]:
# This function can take a list of tool calls, and run them. This is the IF statement!!

def handle_tool_calls(tool_calls):
    results = []
    for tool_call in tool_calls:
        tool_name = tool_call.function.name
        arguments = json.loads(tool_call.function.arguments)
        print(f"Tool called: {tool_name}", flush=True)

        # THE BIG IF STATEMENT!!!
        result = None
        is_find_books = False
        if tool_name == "find_books":
            result = find_books(**arguments)
            is_find_books = True
        elif tool_name == "record_unknown_question_about_book":
            result = record_unknown_question_about_book(**arguments)

        # Convert Book objects to dicts for JSON serialization
        if is_find_books and result:
            content = json.dumps([{"title": b.title, "author": b.author, "year": b.year, "isbns": b.isbns} for b in result])
        else:
            content = json.dumps(result)
            
        results.append({"role": "tool", "content": content, "tool_call_id": tool_call.id})

        books = result if is_find_books else None
    return results, books 

In [ ]:
system_prompt = (
    "You are an expert on artists and famous paintings they have painted. You answer questions on famous painters,"
    "their career, skills and paitings they have painted. But you can asked about literature, books or authors (writers) who wrote some books as well." 

    "If you are asked about literature or books or author that wrote some books, you can use the find_books tool to get a list of books that this author has written. "

    # "If you are asked about the cover image of a book, you can use the get_book_cover_image tool to get the image. "
    "If you don't know the answer about a book or author (writer), use the record_unknown_question_about_book tool to record the question you couldn't answer. "
    "If the user is engaging in discussion, steer them towards getting in touch via email; ask for their email and record it using the record_user_details tool. "
    "With this context, please chat with the user, always staying in character as a specialist in painting."
)


In [ ]:
openai = OpenAI()

def chat(history):
    # Gradio passes (user_question, ai_answer text); use only the user message (single turn).
    history = [{"role":h["role"], "content":h["content"]} for h in history]
    messages = [{"role": "system", "content": system_prompt}] + history

    done = False
    books = None
    while not done:

        # This is the call to the LLM - see that we pass in the tools json

        response = openai.chat.completions.create(model="gpt-4o-mini", messages=messages, tools=tools)

        finish_reason = response.choices[0].finish_reason
        
        # If the LLM wants to call a tool, we do that!
         
        if finish_reason=="tool_calls":
            message = response.choices[0].message
            tool_calls = message.tool_calls
            results, books = handle_tool_calls(tool_calls)
            # print("handle_tool_calls results", results)
            messages.append(message)
            messages.extend(results)
        else:
            done = True
            
    content = response.choices[0].message.content
    content = content if content is not None else "(No response)"
    
    new_history = history + [{"role": "assistant", "content": content}]
    book_image = None
    if books:
        isbn = Book.get_book_isbn(books)
        if isbn:
            data = Book.get_book_cover_image(isbn)
            if data:
                import io
                from PIL import Image as PILImage
                book_image = PILImage.open(io.BytesIO(data))

                
    return new_history, book_image

In [ ]:
# Gradio app: row1 = AI answer (TextBox) + book cover (Image), row2 = user question (TextBox) + Submit button

def put_message_in_chatbot(message, history):
        return "", history + [{"role":"user", "content":message}]


with gr.Blocks() as ui:
    with gr.Row():
        ai_answer = gr.Chatbot(height=500)
        book_image = gr.Image(height=500, interactive=False)
    with gr.Row():
        user_question = gr.Textbox(label="Ask a question about painters or writers and their books", placeholder="e.g. War and Peace")

    # Allow submit by clicking the button or pressing Enter in the user_question textbox
    user_question.submit(fn=put_message_in_chatbot, inputs=[user_question, ai_answer], outputs=[user_question, ai_answer]).then(
        chat, 
        inputs=ai_answer, 
        outputs=[ai_answer, book_image])



In [95]:
ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7868
* To create a public link, set `share=True` in `launch()`.


Tool called: find_books
The Lost Symbol — Dan Brown (2009) ISBN: 9789722524889, 9789955132431
Angels & Demons — Dan Brown (2000) ISBN: 9788086518435, 9789100110567, 9789979774921, 9780552173865, 9780552159760, 9780593055342, 9781416528654, 9781416578741
The Da Vinci Code — Dan Brown (2003) ISBN: 9789584255181, 9788373594210, 9780593055816, 9780552173872, 9780385513227, 9780552171342
Found 3 books for Dan Brown
handle_tool_calls results [{'role': 'tool', 'content': '[{"title": "The Lost Symbol", "author": "Dan Brown", "year": 2009, "isbns": "9789722524889, 9789955132431"}, {"title": "Angels & Demons", "author": "Dan Brown", "year": 2000, "isbns": "9788086518435, 9789100110567, 9789979774921, 9780552173865, 9780552159760, 9780593055342, 9781416528654, 9781416578741"}, {"title": "The Da Vinci Code", "author": "Dan Brown", "year": 2003, "isbns": "9789584255181, 9788373594210, 9780593055816, 9780552173872, 9780385513227, 9780552171342"}]', 'tool_call_id': 'call_U3Sfe8iPATu6cQdUiIVQlMtA'}]
I